# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise - model based cf

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

간단한 surprise 실습

In [9]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from tqdm import tqdm_notebook as tq

In [18]:
data = Dataset.load_builtin('ml-100k',prompt = False)
data.raw_ratings[:100] # user, item, user_review, user_id

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013'),
 ('62', '257', 2.0, '879372434'),
 ('286', '1014', 5.0, '879781125'),
 ('200', '222', 5.0, '876042340'),
 ('210', '40', 3.0, '891035994'),
 ('224', '29', 3.0, '888104457'),
 ('303', '785', 3.0, '879485318'),
 ('122', '387', 5.0, '879270459'),
 ('194', '274', 2.0, '879539794'),
 ('291', '1042', 4.0, '874834944'),
 ('234', '1184', 2.0, '892079237'),
 ('119', '392', 4.0, '886176814'),
 ('167', '486', 4.0, '892738452'),
 ('299', '144', 4.0, '877881320'),
 ('291', '118', 2.0, '874833878'),
 ('308', '1', 4.0, '887736532'),
 ('95', '546', 2.0, '879196566'),
 ('38', '95', 5.0, '892430094'),
 ('102', '768', 2.0, '883748450'),
 ('63', '277', 4.0, '875747401

In [12]:
model = SVD()

### SVD를 활용하여 각각의 measure 측정

In [15]:
cross_validate(model,data,measures = ['rmse','mae'],cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9392  0.9315  0.9357  0.9290  0.9420  0.9355  0.0048  
MAE (testset)     0.7396  0.7332  0.7392  0.7330  0.7406  0.7371  0.0033  
Fit time          9.13    9.18    8.71    9.05    9.96    9.20    0.41    
Test time         0.28    0.27    0.26    0.30    0.27    0.28    0.01    


{'test_rmse': array([0.93916614, 0.93152478, 0.93567461, 0.92898797, 0.94204177]),
 'test_mae': array([0.73964231, 0.73322747, 0.7391848 , 0.73301872, 0.74060978]),
 'fit_time': (9.130003929138184,
  9.176007986068726,
  8.705805540084839,
  9.047016143798828,
  9.956223011016846),
 'test_time': (0.27710580825805664,
  0.2703235149383545,
  0.2624225616455078,
  0.30156826972961426,
  0.268876314163208)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [19]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [21]:
data = Dataset.load_builtin('ml-100k',prompt=False)
raw_data = np.array(data.raw_ratings,dtype=int)

In [22]:
raw_data[:,0] -=1
raw_data[:,1] -=1

In [24]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users +1,n_movies+1)
shape

(943, 1682)

In [30]:
adj_matrix = np.ndarray(shape,dtype = int)
for user_id,movie_id,rting,time in raw_data:
    adj_matrix[user_id][movie_id] =1.
adj_matrix # 1이 있는 위치가 현재 id가 있는 위치이다.

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [33]:
my_id, my_vector = 0,adj_matrix [0]
best_match, best_match_id, best_match_vector =-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector,user_vector)
        if similarity > best_match: # 이경우가 183, 인접행렬을 통해 유사도를 계산하였다.
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:183, Best Match ID:275


### 추천 리스트

In [35]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 <1. and log2 >0.:
        recommend_list.append(i)
print(recommend_list) 

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

## 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
#### 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [38]:
my_id, my_vector = 0,adj_matrix [0]
best_match, best_match_id, best_match_vector = 9999, -1,[]

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector))) # 위에 있는 euclidean 공식 수식화
        if euclidean_dist < best_match: 
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:14.832396974191326, Best Match ID:737


In [39]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 <1. and log2 >0.:
        recommend_list.append(i)
print(recommend_list) 

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산 코드 만들기

In [18]:
def compute_cos_similarity(v1,v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1,v2)
    return dot/(norm1*norm2)

In [44]:
my_id, my_vector = 0,adj_matrix [0]
best_match, best_match_id, best_match_vector = -1, -1,[]

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector,user_vector) #코사인 유사도 함수
        if cos_similarity > best_match: 
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:0.5278586163659506, Best Match ID:915


## 915번은 봤지만 나는 안본 영화 추천

In [45]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 <1. and log2 >0.:
        recommend_list.append(i)
print(recommend_list) 

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험해보았다.

In [47]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix #기존 정보는 1을 넣으면서 봤냐 안봤냐만 알고 있었지만 -> 본 이후에 평가 점수를 반영한다

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [48]:
my_id, my_vector = 0,adj_matrix [0]
best_match, best_match_id, best_match_vector = 9999, -1,[]

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector))) # 위에 있는 euclidean 공식 수식화
        if euclidean_dist < best_match: 
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:55.06359959174482, Best Match ID:737


In [49]:
my_id, my_vector = 0,adj_matrix [0]
best_match, best_match_id, best_match_vector = -1, -1,[]

for user_id,user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector,user_vector) #코사인 유사도 함수
        if cos_similarity > best_match: 
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:0.569065731527988, Best Match ID:915


## 협업 필터링(Collaborative Filtering) - memory based

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [50]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [52]:
data = Dataset.load_builtin('ml-100k',prompt=False)

* KNN을 사용한 협업 필터링

In [53]:
model=KNNBasic() 
cross_validate(model,data,measures=['rmse','mae'],cv =5, n_jobs=4, verbose=True) 

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9857  0.9704  0.9764  0.9831  0.9763  0.9784  0.0054  
MAE (testset)     0.7795  0.7684  0.7715  0.7743  0.7712  0.7730  0.0037  
Fit time          0.65    0.64    0.59    0.55    0.44    0.58    0.07    
Test time         3.38    3.40    3.41    3.35    2.46    3.20    0.37    


{'test_rmse': array([0.98573857, 0.97038572, 0.97637102, 0.98307502, 0.97631092]),
 'test_mae': array([0.77946979, 0.76839996, 0.77150216, 0.77433637, 0.77120902]),
 'fit_time': (0.6462881565093994,
  0.6438088417053223,
  0.5927224159240723,
  0.5540308952331543,
  0.4434654712677002),
 'test_time': (3.3837175369262695,
  3.4000844955444336,
  3.4144670963287354,
  3.3509817123413086,
  2.4621055126190186)}

* SVD를 사용한 협업 필터링

In [54]:
model= SVD()
cross_validate(model,data,measures=['rmse','mae'],cv =5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9445  0.9411  0.9245  0.9342  0.9389  0.9366  0.0069  
MAE (testset)     0.7446  0.7403  0.7288  0.7369  0.7411  0.7383  0.0054  
Fit time          5.99    5.92    5.95    5.98    4.11    5.59    0.74    
Test time         0.21    0.20    0.22    0.20    0.13    0.19    0.03    


{'test_rmse': array([0.94446848, 0.94112281, 0.92448033, 0.93423022, 0.93894047]),
 'test_mae': array([0.74463463, 0.74028722, 0.72884103, 0.73686723, 0.74109707]),
 'fit_time': (5.987718105316162,
  5.915302276611328,
  5.946056127548218,
  5.976809978485107,
  4.114366054534912),
 'test_time': (0.20931291580200195,
  0.20385527610778809,
  0.21625423431396484,
  0.19889354705810547,
  0.12896466255187988)}

* NMF를 사용한 협업 필터링

* SVD++를 사용한 협업 필터링 # run time이 너무 오래걸려 실행해보지 못했다 ㅠ

In [ ]:
model= SVDpp()
cross_validate(model,data,measures=['rmse','mae'],cv =5, n_jobs=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9237  0.9107  0.9172  0.9213  0.9272  0.9200  0.0057  
MAE (testset)     0.7230  0.7136  0.7205  0.7230  0.7278  0.7216  0.0047  
Fit time          686.16  675.43  688.32  673.90  172.86  579.34  203.32  
Test time         12.88   12.85   8.31    12.56   3.16    9.95    3.81    


## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천해보았다.

In [2]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [3]:
data = Dataset.load_builtin('ml-100k',prompt=False)
raw_data = np.array(data.raw_ratings,dtype=int)
raw_data[:,0] -=1
raw_data[:,1] -=1

In [6]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users +1, n_movies +1)
shape

(943, 1682)

In [8]:
adj_matrix = np.ndarray(shape,dtype=int)
for user_id,movie_id,rating, time in raw_data:
    adj_matrix[user_id][movie_id] =rating

In [11]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [12]:
U,S,V = randomized_svd(adj_matrix,n_components=2)
S = np.diag(S)

In [15]:
print(U.shape)
print(S.shape) # 잠재요인을 찾을 수 있는 특이값 vector
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [16]:
np.matmul(np.matmul(U,S),V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

### 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [20]:
my_id, my_vector = 0,U[0]
best_match, best_match_id, best_match_vector = -1, -1,[]

for user_id,user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector,user_vector) #코사인 유사도 함수
        if cos_similarity > best_match: 
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:0.9999942295956324, Best Match ID:235


### Hybrid -> Cos similarity / 56% -> 99%

In [21]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id],adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 <1. and log2 >0.:
        recommend_list.append(i)
print(recommend_list) 

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


### 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [34]:
my_id, my_vector = 0,V.T[0] #V값은 곱으로 Decomposition 했기 때문에 tranpose 한 번 해준다
best_match, best_match_id, best_match_vector = -1, -1,[]

for user_id,user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector,user_vector) #코사인 유사도 함수
        if cos_similarity > best_match: 
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:0.9999999951364145, Best Match ID:1287


In [35]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)
print(recommend_list) 

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [36]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [37]:
A,B,iter = non_negative_factorization(adj_matrix,n_components=2)

C:\Users\vythf\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [42]:
np.matmul(A,B)

array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585167e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275191e-05, 0.00000000e+00],
       ...,
       [1.04485667e+00, 1.17126289e-01, 2.72238757e-01, ...,
        2.34950819e-03, 1.49818804e-05, 0.00000000e+00],
       [1.45758857e+00, 5.42112787e-01, 2.99181695e-01, ...,
        1.61177659e-03, 5.15921394e-03, 5.10771005e-03],
       [2.44733103e+00, 9.41208708e-01, 4.95740038e-01, ...,
        2.56992125e-03, 9.08287324e-03, 8.99389282e-03]])

### 사용자 기반 추천

In [38]:
my_id, my_vector = 0, U[0]  # 내 id가 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1,[]

for user_id,user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector,user_vector) #코사인 유사도 함수
        if cos_similarity > best_match: 
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector 
            
print('Best Match:{}, Best Match ID:{}'.format(best_match,best_match_id)) 

Best Match:0.9999942295956324, Best Match ID:235


In [39]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천

In [44]:
my_id, my_vector = 0, V.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
    if my_id != item_id:
        cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
        best_match = cos_similarity
        best_match_id = item_id
        best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999951364145, Best Match ID: 1287


In [46]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 